# Clues Data Cleaning

The purpose of this notebook is to take the raw JSON files created with scrapy, make them readable in pandas, and export the data to a csv. We begin by cleaning the clues data separately and then merging the correct response data and categories data.



### Reading clues.json

In clues.json, every line is a separate game, and the values are stored in lists. This is an advisable way to store the data, but it was the only way I could find to scrape all the clues and their correct responses in one spider. We make this data readable by pandas by converting all the lists to have equal length and then reading every game in separately, and concatenating all games together. In the raw data, the 'game_id' column is a list of length one, so we multiply that by the number of clues in the game. There is also the possibility that there is a tiebreaker question, which results in the 'value' and 'order_number' liss being one less entry shorter, so in those cases we append 'Tiebreaker' to each list. We then create a base dataframe, and use a for loops to concatenate all other games on top of the base.

In [ ]:
import os
import json
import pandas as pd
import re
import numpy as np
pd.options.display.max_colwidth = 125
pd.options.display.max_rows = 100
os.chdir('C:\\Users\\mhous\\scrap\\JeopardyProject')

In [2]:
#Load first dataset
with open('clues.json') as json_file: 
    raw_data = json.load(json_file) 

In [3]:
#Game_id is a list of 1, so multiply the element of the list by the number of clues in the dataset
for i in range(len(raw_data)):
    raw_data[i]['game_id'] = raw_data[i]['game_id'] * len(raw_data[i]['clue'])

In [4]:
#However, it is possible that the contestants did not have time to answer each question, and so there is are "missing" clues. This function tests if the length of each list is equal
def lengths_check(data):
    id_dict = {}
    for i in range(len(data)):
        len_id = len(data[i]['game_id'])
        len_value = len(data[i]['value'])
        len_order = len(data[i]['order_number'])
        len_clue_id = len(data[i]['clue_id'])
        len_clue = len(data[i]['clue'])
        len_cr = len(data[i]['correct_response'])
        result = len_id == len_value == len_order == len_clue_id == len_clue == len_cr
        id_dict[i] = result
    return id_dict

In [5]:
#Apply the function for the first dataset
raw_data_id_dict = lengths_check(raw_data)
#This code gives the index of games that fail the lengths_test function, so we must append "Tiebreaker" to the value and order_number list
games_to_fix = [k for k,v in raw_data_id_dict.items() if v == False]
games_to_fix

[301, 402, 483, 1153, 2243, 2735, 3868]

In [6]:
#Append "Tiebreaker" to appropriate lists
for i in games_to_fix:
    raw_data[i]['value'].append("Tiebreaker")
    raw_data[i]['order_number'].append('Tiebreaker')

In [7]:
#Double check to make sure everything is compatible 
raw_data_1_id_dict_new = lengths_check(raw_data)
print([k for k,v in raw_data_1_id_dict_new.items() if v == False])

[]


In [8]:
#Create a base dataframe from the first game in the raw data, then append every other game
base = pd.DataFrame(raw_data[0])

for i in range(1, len(raw_data)):
    next_game = pd.DataFrame(raw_data[i])
    base = pd.concat([base, next_game])

### Modifying the Clues Dataframe

We use the 'game_id' column to create two separate columns: 'show_number' and 'date'. show_number acts as a much better index, and date is easier to use when it is in its own column.

We then expand the 'clue_id' column into three columns: 'round', 'row', and 'column'. This is useful because it enables us to use groupby on the new columns.

In [9]:
#Let's split the game_id column into more descriptive columns. This also allows us to use show_number as an index
base['game_id'] = base['game_id'].astype("string")
base['show_number'] = base['game_id'].str.split(' ').str[1].str.strip('#')
day = base['game_id'].str.split(' ').str[5].str.strip(',')
month = base['game_id'].str.split(' ').str[4]
year =  base['game_id'].str.split(' ').str[6]
date = day + ' ' + month + ' ' + year
base['date'] = pd.to_datetime(date)
base.head()

,game_id,value,order_number,clue_id,clue,correct_response,show_number,date
0,"Show #7943 - Wednesday, March 6, 2019",$200,8,clue_J_1_1,"In an effort to preserve this 800-year-old Italian structure, really heavy counterweights were installed",the Leaning Tower of Pisa,7943,2019-03-06
1,"Show #7943 - Wednesday, March 6, 2019",$200,21,clue_J_2_1,Married to GB: TB12,Tom Brady,7943,2019-03-06
2,"Show #7943 - Wednesday, March 6, 2019",$200,13,clue_J_3_1,He rids your house of vermin,an exterminator,7943,2019-03-06
3,"Show #7943 - Wednesday, March 6, 2019",$200,1,clue_J_4_1,This tragic Shakespeare title king has a cold night reflect on filial ingratitude,King Lear,7943,2019-03-06
4,"Show #7943 - Wednesday, March 6, 2019",$200,3,clue_J_5_1,"In dry air, the speed of sound is 1,086 FPS, short for this",feet per second,7943,2019-03-06


In [10]:
#Let's also split the clue_id column into more descriptive columns so that they can be matched to the categories dataset
base['round'] = base['clue_id'].str.split('_').str[1]
base['column'] = base['clue_id'].str.split('_').str[2]
base['row'] = base['clue_id'].str.split('_').str[3]
base = base.drop(['clue_id'], axis=1)
base.head()

,game_id,value,order_number,clue,correct_response,show_number,date,round,column,row
0,"Show #7943 - Wednesday, March 6, 2019",$200,8,"In an effort to preserve this 800-year-old Italian structure, really heavy counterweights were installed",the Leaning Tower of Pisa,7943,2019-03-06,J,1,1
1,"Show #7943 - Wednesday, March 6, 2019",$200,21,Married to GB: TB12,Tom Brady,7943,2019-03-06,J,2,1
2,"Show #7943 - Wednesday, March 6, 2019",$200,13,He rids your house of vermin,an exterminator,7943,2019-03-06,J,3,1
3,"Show #7943 - Wednesday, March 6, 2019",$200,1,This tragic Shakespeare title king has a cold night reflect on filial ingratitude,King Lear,7943,2019-03-06,J,4,1
4,"Show #7943 - Wednesday, March 6, 2019",$200,3,"In dry air, the speed of sound is 1,086 FPS, short for this",feet per second,7943,2019-03-06,J,5,1


In [11]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255263 entries, 0 to 59
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   game_id           255263 non-null  string        
 1   value             255263 non-null  object        
 2   order_number      255263 non-null  object        
 3   clue              255263 non-null  object        
 4   correct_response  255263 non-null  object        
 5   show_number       255263 non-null  object        
 6   date              255263 non-null  datetime64[ns]
 7   round             255263 non-null  object        
 8   column            250976 non-null  object        
 9   row               250976 non-null  object        
dtypes: datetime64[ns](1), object(8), string(1)
memory usage: 21.4+ MB


### Reading and Modifying responses.json

In responses.json each line is its own clue and the entries are 'correct_response', 'value', 'order_number', 'correct', 'incorrect' and 'game_id'. While we already have the correct response stored in clues.json, responses.json stores who answered the questions correctly in lists, which is not possible in the clues spider. 

We begin by creating the 'show_number' column based on 'game_id', which will help us merge this dataframe with the clues dataframe. We then convert the lists of 'correct' and 'incorrect' into strings, which makes it more readable. We also create columns based on the number of people who answered correctly, incorrectly, or did not know. 

In order to merge these datasets, we must merge on 'show_number', 'correct_response', 'value', and'order_number'. 'show_number' and 'corrrect_response' pose no troubles, but we must modify things for 'value' and 'order_number'. Since the data for correct responses comes from a separate page, the notation is slightly different than for the clues data. The page for correct responses does not include the 'DD: ' to indicate a daily double, so we create a 'value_merge' column in the clues dataframe that replaces the 'DD: ' with ''. Additionally, tiebreakers also cause trouble in merging these two dataframes because in responses.json their 'value' and 'order_number' are NaN. To fix this, we find all tiebreaker questions from the clues dataframe (there's only 7), create masks in order to identify those clues in the responses dataframe, then replace their 'value_merge' and 'order_number' values with "Tiebreaker" in order for the dataframes to merge properly.

In [13]:
responses = pd.read_json('responses.json')
responses['game_id'] = responses['game_id'].astype("string")
responses['show_number'] = responses['game_id'].str.split(' ').str[1].str.strip('#')
responses = responses.rename({'right': 'correct', 'wrong': 'incorrect'}, axis=1)
responses = responses.drop(['game_id'], axis=1)
responses.head()

,correct_response,value,order_number,correct,incorrect,show_number
0,England (Great Britain),$200,22,[Geoffrey],[],3966
1,Les Miserables,$200,1,[Geoffrey],[],3966
2,Home Depot,$200,2,[Geoffrey],[Kristin],3966
3,birch,$200,20,[Kristin],[],3966
4,Wisconsin,$200,4,[Brandon],[],6089


In [14]:
responses['correct'] = [', '.join(map(str, l)) for l in responses['correct']]
responses['incorrect'] = [', '.join(map(str, l)) for l in responses['incorrect']]

In [15]:
n_didnt_know = []
for i in responses['incorrect']:
    if 'Triple Stumper' in i:
        n_didnt_know.append('3')
    elif i == "":
        n_didnt_know.append(0)
    elif i.count(',') ==0:
        n_didnt_know.append(1)
    elif i.count(',') == 1:
        n_didnt_know.append(2)
    elif i.count(',') == 2:
        n_didnt_know.append(3)
print("Lengths equal:", len(n_didnt_know) == len(responses))

Lengths equal: True


In [16]:
n_correct = []
for i in responses['correct']:
    if i == "":
        n_correct.append(0)
    elif i.count(',') ==0:
        n_correct.append(1)
    elif i.count(',') == 1:
        n_correct.append(2)
    elif i.count(',') == 2:
        n_correct.append(3)
print("Lengths equal:", len(n_correct) == len(responses))

Lengths equal: True


In [17]:
n_incorrect = []
for i in responses['incorrect']:
    if i == "":
        n_incorrect.append(0)
    elif (i.count(',') == 0) & ('Triple Stumper' in i):
        n_incorrect.append(0)
    elif (i.count(',') == 1) & ('Triple Stumper' in i):
        n_incorrect.append(1)
    elif (i.count(',') == 2) & ('Triple Stumper' in i):
        n_incorrect.append(2)
    elif (i.count(',') == 3) & ('Triple Stumper' in i):
        n_incorrect.append(3)
    elif i.count(',') == 0:
        n_incorrect.append(1)
    elif (i.count(',') == 1):
        n_incorrect.append(2)
    elif (i.count(',') == 2):
        n_incorrect.append(3)
print("Lengths equal: ", len(n_incorrect) == len(responses))

Lengths equal:  True


In [18]:
responses['n_didnt_know'] = n_didnt_know
responses['n_correct'] = n_correct
responses['n_incorrect'] = n_incorrect

In [19]:
responses.columns

Index(['correct_response', 'value', 'order_number', 'correct', 'incorrect',
       'show_number', 'n_didnt_know', 'n_correct', 'n_incorrect'],
      dtype='object')

In [20]:
# Change the column order to be more readable
cols = ['show_number','value', 'order_number', 'correct_response', 'correct', 'incorrect', 'n_correct', 'n_incorrect',  'n_didnt_know']
responses = responses.reindex(columns=cols)
responses = responses.dropna()
responses.head()

In [22]:
#Create 'value_merge' column that enables the columns to be merged properly by replacing the 'DD: ' which is found in base with ''
base['value_merge'] = base['value'].str.replace('DD: ', '')
responses['value_merge'] = responses['value']

In [23]:
#Find all tiebreaker questions
base[base['order_number'] == 'Tiebreaker']

,game_id,value,order_number,clue,correct_response,show_number,date,round,column,row,value_merge
58,"Show #4150 - Friday, September 20, 2002",Tiebreaker,Tiebreaker,"Hogwarts headmaster, he's considered by many to be the greatest wizard alive",Professor Dumbledore,4150,2002-09-20,TB,NaN,NaN,Tiebreaker
61,"Show #8023 - Wednesday, June 26, 2019",Tiebreaker,Tiebreaker,"Types of it you could find in Boston Harbor on December 16, 1773 included Souchong & Bohea",tea,8023,2019-06-26,TB,NaN,NaN,Tiebreaker
59,"Show #8039 - Thursday, July 18, 2019",Tiebreaker,Tiebreaker,This noble gas is the second-lightest element,helium,8039,2019-07-18,TB,NaN,NaN,Tiebreaker
61,"Show #6370 - Friday, May 4, 2012",Tiebreaker,Tiebreaker,"Although he doesn't actually appear in ""1984"", his presence is everywhere--on posters, coins & telescreens",Big Brother,6370,2012-05-04,TB,NaN,NaN,Tiebreaker
61,"Show #6895 - Friday, August 1, 2014",Tiebreaker,Tiebreaker,The battles of Shiloh & Collierville were fought in this state,Tennessee,6895,2014-08-01,TB,NaN,NaN,Tiebreaker
61,"Show #5332 - Tuesday, November 13, 2007",Tiebreaker,Tiebreaker,A Longfellow poem & a Lillian Hellman play about a girls' boarding school share this timely title,The Children's Hour,5332,2007-11-13,TB,NaN,NaN,Tiebreaker
59,"Show #7709 - Thursday, March 1, 2018",Tiebreaker,Tiebreaker,Her April decision to call a snap parliamentary election proved less than brilliant on June 8,(Theresa) May,7709,2018-03-01,TB,NaN,NaN,Tiebreaker


In [24]:
#Create masks for each tiebreaker question
mask1 = responses['correct_response'] == 'Professor Dumbledore'
mask2 = (responses['correct_response'] == 'tea') & (responses['value'] == 'FJ')
mask3 = (responses['correct_response'] == 'helium') & (responses['value'] == 'FJ') & (responses['show_number'] == '8039')
mask4 = (responses['correct_response'] == 'Big Brother') & (responses['value'] == 'FJ')
mask5 = (responses['correct_response'] == 'Tennessee') & (responses['value'] == 'FJ') & (responses['show_number'] == '6895')
mask6 = (responses['correct_response'] == "The Children's Hour") & (responses['value'] == 'FJ') 
mask7 = (responses['correct_response'] == '(Theresa) May') & (responses['value'] == 'FJ') 
masks = [mask1, mask2, mask3, mask4, mask5, mask6, mask7]
#Modify the values in responses so they can be merged
for i in masks:
    responses.loc[i, 'value_merge'] = 'Tiebreaker'
    responses.loc[i, 'order_number'] = 'Tiebreaker'

In [25]:
df = base.merge(responses, on = ['show_number', 'correct_response', 'value_merge', 'order_number'], how = 'left')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255263 entries, 0 to 255262
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   game_id           255263 non-null  string        
 1   value_x           255263 non-null  object        
 2   order_number      255263 non-null  object        
 3   clue              255263 non-null  object        
 4   correct_response  255263 non-null  object        
 5   show_number       255263 non-null  object        
 6   date              255263 non-null  datetime64[ns]
 7   round             255263 non-null  object        
 8   column            250976 non-null  object        
 9   row               250976 non-null  object        
 10  value_merge       255263 non-null  object        
 11  value_y           255263 non-null  object        
 12  correct           255263 non-null  object        
 13  incorrect         255263 non-null  object        
 14  n_co

In [27]:
#Drop the 'value_y' column and rename 'value_x' with 'value'
df = df.drop(['value_y'], axis=1)
df= df.rename(columns = {'value_x':'value'})

### Reading categories.json

categories.json is especially weird, but workable. One intricacy of this data is that there is always a tiebreaker category, even for games that did not end in tiebreakers. This is because we still need the category for tiebreakers, and there was no way to only include that data specifically for tiebreaker games. When scraping the data, categories are scraped into a list, and then gets assigned to its correct column by its index location. Since the list can have a length of either 13 or 14, we assign the final jeopardy category to the 13th index and the tiebreaker to the last element of the index. What this means is that for games without tiebreakers, the tiebreaker category will be the same as the final jeopardy category, but for games that end in tiebreakers their tiebreaker category is correctly labelled. Ultimately, there is lots of redundant data, but that data 'disappears' when merging it to the main clues data.

Since each value is a list of length one, we begin by turning each list into a string and remove the '' enclosing it. We then melt the dataframe so that the columns become rows. We break the new 'variable' column that contains the location of each category down into a 'round' and 'column' columns. Finally we make the data uppercase and rename the columns so they can be properly used in a merge. 

In [28]:
#load categories_data
with open('categories.json') as json_file: 
    categories_data = json.load(json_file) 

In [29]:
#Since each element is a list of one, remove the list and leave the string for every key
for i in range(len(categories_data)):
    categories_data[i]['game_id'] = str(categories_data[i]['game_id']).strip('[]')
    categories_data[i]['category_j_1'] = str(categories_data[i]['category_j_1']).strip('[]')
    categories_data[i]['category_j_2'] = str(categories_data[i]['category_j_2']).strip('[]')
    categories_data[i]['category_j_3'] = str(categories_data[i]['category_j_3']).strip('[]')
    categories_data[i]['category_j_4'] = str(categories_data[i]['category_j_4']).strip('[]')
    categories_data[i]['category_j_5'] = str(categories_data[i]['category_j_5']).strip('[]')
    categories_data[i]['category_j_6'] = str(categories_data[i]['category_j_6']).strip('[]')
    categories_data[i]['category_dj_1'] = str(categories_data[i]['category_dj_1']).strip('[]')
    categories_data[i]['category_dj_2'] = str(categories_data[i]['category_dj_2']).strip('[]')
    categories_data[i]['category_dj_3'] = str(categories_data[i]['category_dj_3']).strip('[]')
    categories_data[i]['category_dj_4'] = str(categories_data[i]['category_dj_4']).strip('[]')
    categories_data[i]['category_dj_5'] = str(categories_data[i]['category_dj_5']).strip('[]')
    categories_data[i]['category_dj_6'] = str(categories_data[i]['category_dj_6']).strip('[]')  
    categories_data[i]['category_fj'] = str(categories_data[i]['category_fj']).strip('[]')  
    categories_data[i]['category_tb'] = str(categories_data[i]['category_tb']).strip('[]')  

In [30]:
#Make pandas dataframe of this data
categories = pd.DataFrame(categories_data)
categories.head()

,game_id,category_j_1,category_j_2,category_j_3,category_j_4,category_j_5,category_j_6,category_dj_1,category_dj_2,category_dj_3,category_dj_4,category_dj_5,category_dj_6,category_fj,category_tb
0,"'Show #7943 - Wednesday, March 6, 2019'",'HISTORIC STRUCTURES','ALPHANUMERIC ATHLETES',"'IN THE ""MI""DDLE'",'COLD PLAY','SPEED OF SOUND','YOLO','HIGH-SCORING SCRABBLE WORDS','EX-CREATURES','AROUND THE PACIFIC','19th CENTURY PEOPLE','TALK LIKE AN OLD SAILOR',"'THIS IS ""MY"" SONG'",'WORD ORIGINS','WORD ORIGINS'
1,"'Show #6089 - Thursday, February 17, 2011'",'THE 50 STATES','DISNEYLAND RIDES & ATTRACTIONS','CHEMISTRY','TEENS & POETRY','SODA',"'STRIVING FOR AN ""F""'",'HISTORY','GLEE SONGS','WHAT KIDS ARE READING THESE DAYS','BOARDING SCHOOL','PATENTS','WORDS & PHRASES','EVENTS OF 2010','EVENTS OF 2010'
2,"'Show #4281 - Monday, March 24, 2003'",'SCIENCE & NATURE','MEET THE BEATLE','ABBREV.','FRENCH RULER NICKNAMES',"""LET'S BUILD A CASINO!""","""THERE'S A SUCKER...""",'BOOKS OF THE BIBLE','HOLLYWOOD MIDDLE INITIALS','PRECOLUMBIAN LIFE',"""CONTINENTS' LOWEST POINTS""",'20th CENTURY WOMEN',"'""O"" BABY!'",'THE 7 WONDERS OF THE ANCIENT WORLD','THE 7 WONDERS OF THE ANCIENT WORLD'
3,"'Show #4089 - Thursday, May 16, 2002'",'THE PRICE IS','WRITE','CON-CENTRATION','HARD SHARKS',"""MATCH GAME '73""",'MISSING LINKS','MR. PRIME MINISTER','COLLEGE COLLAGE','BILL MURRAY MOVIE QUOTES','DOCTORS','NAMES ON THE MAP',"'""CHECK"" MATE'",'MILITARY UNITS','MILITARY UNITS'
4,"'Show #3966 - Monday, November 26, 2001'",'SUDDENLY SUDAN','GOOD TOMES','STORE TREK','TREES COMPANY','MELROSE FACE',"'""FOOL"" HOUSE'",'CLASSICAL MUSIC','UNIVERSITY STATE','ODDS & ENDS','AT THE MOVIES','HISTORIC TABLOID HEADLINES','PORTMANTEAU WORDS','WORLD CITIES','WORLD CITIES'


In [31]:
#Create show_number column to more easily identify each game
categories['show_number'] = categories['game_id'].str.split(' ').str[1].str.strip('#')

In [32]:
#Strip the '' on each string
for i in categories.columns:
    categories[i] = categories[i].str.replace(r"['']", '')

In [33]:
#Drop the game_id column for easier melting
categories = categories.drop(['game_id'], axis=1)

In [34]:
#Melt the dataset so that there is a column for show number, a column to describe the category's location and round, and then the value for each categry
categories_melt = categories.melt(id_vars = ['show_number'], value_vars = ['category_j_1', 'category_j_2', 'category_j_3',
       'category_j_4', 'category_j_5', 'category_j_6', 'category_dj_1',
       'category_dj_2', 'category_dj_3', 'category_dj_4', 'category_dj_5',
       'category_dj_6', 'category_fj', 'category_tb'])

In [35]:
categories_melt.head()

,show_number,variable,value
0,7943,category_j_1,HISTORIC STRUCTURES
1,6089,category_j_1,THE 50 STATES
2,4281,category_j_1,SCIENCE & NATURE
3,4089,category_j_1,THE PRICE IS
4,3966,category_j_1,SUDDENLY SUDAN


In [36]:
#create a round and column category from the variable column, then drop the variable column
categories_melt['round'] = categories_melt['variable'].str.split('_').str[1]
categories_melt['column'] = categories_melt['variable'].str.split('_').str[2]
categories_melt = categories_melt.drop(['variable'], axis=1)
categories_melt

,show_number,value,round,column
0,7943,HISTORIC STRUCTURES,j,1
1,6089,THE 50 STATES,j,1
2,4281,SCIENCE & NATURE,j,1
3,4089,THE PRICE IS,j,1
4,3966,SUDDENLY SUDAN,j,1
...,...,...,...,...
59915,7928,POETS,tb,NaN
59916,7929,COLORFUL GEOGRAPHY,tb,NaN
59917,7930,19th CENTURY INVENTORS,tb,NaN
59918,7931,BRITISH AUTHORS,tb,NaN


In [37]:
#Make the round uppercase so it can merge with the clues dataset, as well as rename the 'value' column to 'category'
categories_melt['round'] = categories_melt['round'].str.upper()
categories_melt = categories_melt.rename(columns = {"value": "category"})
categories_melt.head()

,show_number,category,round,column
0,7943,HISTORIC STRUCTURES,J,1
1,6089,THE 50 STATES,J,1
2,4281,SCIENCE & NATURE,J,1
3,4089,THE PRICE IS,J,1
4,3966,SUDDENLY SUDAN,J,1


In [38]:
#Merge data 
full_data = df.merge(categories_melt, on = ['show_number', 'round', 'column'])
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255263 entries, 0 to 255262
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   game_id           255263 non-null  string        
 1   value             255263 non-null  object        
 2   order_number      255263 non-null  object        
 3   clue              255263 non-null  object        
 4   correct_response  255263 non-null  object        
 5   show_number       255263 non-null  object        
 6   date              255263 non-null  datetime64[ns]
 7   round             255263 non-null  object        
 8   column            250976 non-null  object        
 9   row               250976 non-null  object        
 10  value_merge       255263 non-null  object        
 11  correct           255263 non-null  object        
 12  incorrect         255263 non-null  object        
 13  n_correct         255263 non-null  int64         
 14  n_in

### Finishing Touches

We now have a dataframe where every row is a clue that gives identifying information about its location on the board and episode, the actual text and correct response, who answered it correctly/incorrectly and how many answered correctly/incorrectly. We finalize this data by removing the 'value_merge' column, and reordering the columns so that the order of the columns makes more sense. Finally we export the full dataframe and the categories_melt dataframe to a csv for further analysis. 

In [39]:
full_data = full_data.drop(['value_merge'], axis=1)

In [40]:
full_data.columns

Index(['game_id', 'value', 'order_number', 'clue', 'correct_response',
       'show_number', 'date', 'round', 'column', 'row', 'correct', 'incorrect',
       'n_correct', 'n_incorrect', 'n_didnt_know', 'category'],
      dtype='object')

In [41]:
cols = ['game_id', 'show_number', 'date', 'value', 'round', 'category', 'column', 'row', 'order_number', 'clue', 'correct_response',
        'correct', 'incorrect', 'n_correct', 'n_incorrect', 'n_didnt_know']
full_data = full_data.reindex(columns=cols)

In [42]:
full_data.head()

,game_id,show_number,date,value,round,category,column,row,order_number,clue,correct_response,correct,incorrect,n_correct,n_incorrect,n_didnt_know
0,"Show #7943 - Wednesday, March 6, 2019",7943,2019-03-06,$200,J,HISTORIC STRUCTURES,1,1,8,"In an effort to preserve this 800-year-old Italian structure, really heavy counterweights were installed",the Leaning Tower of Pisa,Tim,,1,0,0
1,"Show #7943 - Wednesday, March 6, 2019",7943,2019-03-06,$400,J,HISTORIC STRUCTURES,1,2,9,"Before billeting the RAF in World War II, Blickling Estate was the birthplace of this second wife of Henry VIII",Anne Boleyn,Eric,,1,0,0
2,"Show #7943 - Wednesday, March 6, 2019",7943,2019-03-06,$600,J,HISTORIC STRUCTURES,1,3,10,"Later famous as a rock concert venue, Tokyo's Nippon Budokan Hall was built to host this martial art at the 1964 Olympics",judo,Tim,,1,0,0
3,"Show #7943 - Wednesday, March 6, 2019",7943,2019-03-06,$800,J,HISTORIC STRUCTURES,1,4,11,"""Scholars"" know the Knights Hospitaller had a palace for the grand master on this large Greek island",Rhodes,,"Dana, Tim, Triple Stumper",0,2,3
4,"Show #7943 - Wednesday, March 6, 2019",7943,2019-03-06,$1000,J,HISTORIC STRUCTURES,1,5,12,Herod built this Israel mountain fortress; later the Romans had trouble taking it from the Zealots,Masada,Dana,,1,0,0


In [43]:
full_data.to_csv(r'C:\\Users\\mhous\\scrap\\JeopardyProject\\clues.csv', index = False)

In [44]:
categories_melt.to_csv(r'C:\\Users\\mhous\\scrap\\JeopardyProject\\categories.csv', index = False)